<a href="https://colab.research.google.com/github/AftabUdaipurwala/Deeplearning/blob/main/Deep_Learning_Gradient_Descent_Problem.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
import cv2
import numpy
import pandas as pd
import numpy as np
from sklearn import linear_model
from google.colab import drive
import matplotlib.pyplot as plt
%matplotlib inline
import matplotlib
matplotlib.rcParams['figure.figsize']=(20,10)
from scipy import stats
import warnings
warnings.filterwarnings("ignore")
import seaborn as sns
from google.colab.patches import cv2_imshow
import tensorflow as tf
from tensorflow import keras



In [4]:
drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
!ls '/content/drive/MyDrive/py-master/DeepLearningML/6_gradient_descent/'

6_gradient_descent.ipynb  insurance_data.csv  nn.png


In [6]:
df = pd.read_csv('/content/drive/MyDrive/py-master/DeepLearningML/6_gradient_descent/insurance_data.csv')
df.head()

,age,affordibility,bought_insurance
0,22,1,0
1,25,0,0
2,47,1,1
3,52,0,0
4,46,1,1


In [7]:
# scaling the data for preprocessing 
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df[['age','affordibility']], df.bought_insurance, test_size=0.2, random_state=123)

In [8]:
len(x_test)

6

In [9]:
# scaling age to improve the machine learning model performance
x_train_scaled = x_train.copy()
x_train_scaled.age = x_train_scaled['age']/100
x_test_scaled = x_test.copy()
x_test_scaled.age = x_test_scaled['age']/100

In [10]:
# trying to build the model using simple neural netwrok
# Kernal_initializer is for initializing weights (i.e. m in y=mx+c) and  bias_initializer is for initializing bias (i.e. C in y=mx+c)
model = keras.Sequential([
                          keras.layers.Dense(1, input_shape=(2,), activation='sigmoid', kernel_initializer='ones', bias_initializer='zeros')
])

model.compile(
    optimizer ='adam',
    loss ='binary_crossentropy',
    metrics =['accuracy']
)

model.fit(x_train_scaled,y_train, epochs=5000)

Streaming output truncated to the last 5000 lines.
1/1 [==============================] - 0s 5ms/step - loss: 0.5420 - accuracy: 0.9091
Epoch 2502/5000
1/1 [==============================] - 0s 13ms/step - loss: 0.5419 - accuracy: 0.9091
Epoch 2503/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.5419 - accuracy: 0.9091
Epoch 2504/5000
1/1 [==============================] - 0s 4ms/step - loss: 0.5419 - accuracy: 0.9091
Epoch 2505/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.5418 - accuracy: 0.9091
Epoch 2506/5000
1/1 [==============================] - 0s 8ms/step - loss: 0.5418 - accuracy: 0.9091
Epoch 2507/5000
1/1 [==============================] - 0s 3ms/step - loss: 0.5417 - accuracy: 0.9091
Epoch 2508/5000
1/1 [==============================] - 0s 6ms/step - loss: 0.5417 - accuracy: 0.9091
Epoch 2509/5000
1/1 [==============================] - 0s 12ms/step - loss: 0.5417 - accuracy: 0.9091
Epoch 2510/5000
1/1 [==============================] -

In [11]:
model.evaluate(x_test_scaled, y_test) # checking model on test data

1/1 [==============================] - 0s 395ms/step - loss: 0.4896 - accuracy: 0.8333


[0.48964834213256836, 0.8333333134651184]

In [12]:
coef,intercept = model.get_weights() # checking weights n bais final values
coef, intercept

(array([[3.9699857],
        [1.6556505]], dtype=float32), array([-2.9109623], dtype=float32))

In [13]:
# building model from scratch in python
def sigmoid(x):
  import math
  return 1/(1+math.exp(-x))
  

In [14]:
def prediction_function(age, affordibility):
  weighted_sum = coef[0]*age+coef[1]*affordibility + intercept
  return sigmoid(weighted_sum)


In [15]:
prediction_function(0.47,1)

0.6480734154166167

In [16]:
prediction_function(0.18,1)

0.3680213864701229

In [17]:
def log_loss(y_true,y_predicted):
  epsilon = 1e-15
  y_predicted_new = [max(i,epsilon) for i in y_predicted]
  y_predicted_new = [min(i,1-epsilon) for i in y_predicted_new]
  y_predicted_new = np.array(y_predicted_new)
  return -np.mean(y_true*np.log(y_predicted_new)+(1-y_true)*np.log(1-y_predicted_new))

In [18]:
# Making sigmoid function using numpy for faster calculation purposes
def sigmoid_numpy(x):
  return 1/(1+np.exp(-x))

sigmoid_numpy(np.array([12,0,1]))
  

array([0.99999386, 0.5       , 0.73105858])

In [21]:
# define gradiant descent function
def gradiant_descent(age,affordibility,y_true,epochs,loss_threshold):
  # w1,w2,bias
  w1=w2=1
  bias =0
  rate =0.5
  n= len(age)

  for i in range(epochs):
    weighted_sum= w1*age+w2*affordibility+bias
    y_predicted = sigmoid_numpy(weighted_sum)
    loss = log_loss(y_true,y_predicted)
    w1d = (1/n)*np.dot(np.transpose(age),(y_predicted-y_true))
    w2d = (1/n)*np.dot(np.transpose(affordibility),(y_predicted-y_true))
    bias_d = np.mean(y_predicted-y_true)
    w1 = w1-rate*w1d
    w2 = w2-rate*w2d
    bias = bias-rate*bias_d
    print(f'Epoch: {i}, w1:{w1} w2:{w2} bias:{bias}, loss{loss}')

    if loss<=loss_threshold:
      break

  return w1, w2, bias


In [22]:
gradiant_descent(x_train_scaled['age'], x_train_scaled['affordibility'],y_train,1000,0.4631)

Epoch: 0, w1:0.96118696347786 w2:0.9273184171082597 bias:-0.15533429441307403, loss0.7871362376444705
Epoch: 1, w1:0.9298653887425002 w2:0.8678493910667062 bias:-0.2899687775893781, loss0.7296902051317885
Epoch: 2, w1:0.9055231798414987 w2:0.820973084627118 bias:-0.4054698987199312, loss0.6879019263184873
Epoch: 3, w1:0.8874120473860106 w2:0.7854964564652974 bias:-0.5039886140661315, loss0.6582663072930622
Epoch: 4, w1:0.8746820119420499 w2:0.7599320377726334 bias:-0.5878990722823709, loss0.6375591541343314
Epoch: 5, w1:0.8664871867707278 w2:0.7427311708850739 bias:-0.6595244536207684, loss0.62314398593432
Epoch: 6, w1:0.8620503623043838 w2:0.7324346086043095 bias:-0.7209758496331984, loss0.6130340755743863
Epoch: 7, w1:0.8606924686834018 w2:0.7277480047994387 bias:-0.774083700155989, loss0.6058136648231096
Epoch: 8, w1:0.8618388899880226 w2:0.727565921781416 bias:-0.8203884264327516, loss0.6005108201595626
Epoch: 9, w1:0.8650132053116407 w2:0.7309666735894639 bias:-0.861162140819556, 

(3.8697535784602803, 1.8624657756437473, -3.0788524236218504)

In [23]:
coef, intercept

(array([[3.9699857],
        [1.6556505]], dtype=float32), array([-2.9109623], dtype=float32))